# Advanced technique of missing value imputation:

# Multiple Imputation by Chained Equations (MICE)


Models each variable with the missing values of a function of other variables in a round-robin fashion.

In [48]:
# import necessary libraries 
import pandas as pd 
import seaborn as sb 
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer

In [49]:
# loading the dataset 
df = sb.load_dataset('titanic') 
# displaying the first 5 rows of the dataset 
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [50]:
# checking the missing values in the dataset 
df.isnull().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [51]:
# dropping deck column as it has too many missing values 
df.drop('deck',axis=1,inplace=True)

In [52]:
# preparing for imputation - encoding categorical variables
from sklearn.preprocessing import LabelEncoder 
cols_to_encode = ['sex','who','embarked','embark_town','alive'] 
# initializing a dictionary to store the label encoders 
encoders = {} 
for col in cols_to_encode:
    le = LabelEncoder() 
    df[col] = le.fit_transform(df[col].astype(str)) # storing the encoder for each column
    encoders[col] = le

encoders

{'sex': LabelEncoder(),
 'who': LabelEncoder(),
 'embarked': LabelEncoder(),
 'embark_town': LabelEncoder(),
 'alive': LabelEncoder()}

In [53]:
# imputing the missing values using MICE 
imputer = IterativeImputer(max_iter=20) 
cols_to_impute = ['age','embarked','embark_town']
for col in cols_to_impute:
    df[col] = imputer.fit_transform(df[[col]]) 
# checking for missing values again 
df.isnull().sum().sort_values(ascending=False)

survived       0
pclass         0
sex            0
age            0
sibsp          0
parch          0
fare           0
embarked       0
class          0
who            0
adult_male     0
embark_town    0
alive          0
alone          0
dtype: int64

In [54]:
# Inverse transforming the encoded columns to their original values 
for col in cols_to_encode:
    df[col] = encoders[col].inverse_transform(df[col].round().astype(int)) 

# displaying the first 20 rows of the dataset after inverse transformation 
df.head(20)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,embark_town,alive,alone
0,0,3,male,22.000000,1,0,7.2500,S,Third,man,True,Southampton,no,False
1,1,1,female,38.000000,1,0,71.2833,C,First,woman,False,Cherbourg,yes,False
2,1,3,female,26.000000,0,0,7.9250,S,Third,woman,False,Southampton,yes,True
3,1,1,female,35.000000,1,0,53.1000,S,First,woman,False,Southampton,yes,False
4,0,3,male,35.000000,0,0,8.0500,S,Third,man,True,Southampton,no,True
5,0,3,male,29.699118,0,0,8.4583,Q,Third,man,True,Queenstown,no,True
6,0,1,male,54.000000,0,0,51.8625,S,First,man,True,Southampton,no,True
7,0,3,male,2.000000,3,1,21.0750,S,Third,child,False,Southampton,no,False
8,1,3,female,27.000000,0,2,11.1333,S,Third,woman,False,Southampton,yes,False
9,1,2,female,14.000000,1,0,30.0708,C,Second,child,False,Cherbourg,yes,False
